In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import joblib

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks

import os, sys

# Helpers
abspath = os.path.abspath
dirname = os.path.dirname
sep = os.sep
file_ = os.getcwd()

ml_folder = dirname(file_)
sys.path.append(ml_folder)

from src.utils import mining_data_tb as md
from src.utils import visualization_tb as vi
from src.utils import folder_tb as fo
from src.utils import models as mo


import warnings

warnings.filterwarnings("ignore")

In [2]:
# As the data variables are coded (for instance, "RIAGENDR" is Gender), we first need to load the variable descriptions. For that, we will create an object with all the info and methods to change names whenever necessary

# 1) We create the object
vardata = md.variables_data()

# 2) We load the info
vardata_path = "data" + sep + "6_variables" + sep + "0_final_variables.csv"
vardata.load_data(2, vardata_path)

# Now we can load the actual dataset we will be using for the ml models

# 1) Create object
dataset = md.dataset()

# 2) Load data
folders = ["1_demographics", "2_dietary", "3_examination", "4_laboratory", "5_questionnaire"]
columns_correction = {
            "WTDRD1_x" : "WTDRD1",
            "WTDR2D_x" : "WTDR2D",
            "DRABF_x" : "DRABF",
            "DRDINT_x" : "DRDINT",
            "WTSAF2YR_x" : "WTSAF2YR",
            "LBXHCT_x" : "LBXHCT"
        }
dataset.load_data(2, folders, columns_correction)

# As we can see, it is quite wide. Let's filter by the columns we will actually be using (using our magnificent object)
# We will start by trying to predict Asthma
features = ["MCQ160H", "RIAGENDR", "RIDAGEYR", "DR1TCHOL", "DR1TTFAT", "DR1TSFAT", "DR1TSUGR", "DR2TCHOL", "DR2TTFAT", "DR2TSFAT", "DR2TSUGR", "BPXDI1", "BPXSY1", "BMXWT", "DXDTOPF", "BMXWAIST", "LBXTR", "LBXTC", "LBXSGL"]

dataset.filter_columns(features, inplace = True)

features_names = vardata.vars_descr_detector(list(dataset.df.columns), 40, True)

dataset.df = dataset.df.drop(["LBXTR", "DXDTOPF"], axis = 1)
dataset.df = dataset.df.dropna()

In [3]:
# Prepare data for the ML models
# We won't scale or balance the data for this first round
dataset.model_data(split = .2, cv = 3, epochs = 1)

# Choose models for test against each other
# Same seed of all of them
seed = 42

log1 = LogisticRegression(n_jobs = -1, random_state = seed)
log2 = LogisticRegression(n_jobs = -1, max_iter = 300, random_state = seed)

models = [log1, log2]

# Models comparison
# Ensemble models
ensembler = mo.model_ensembler(models)
# Load data
ensembler.load_data(dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test, features_names, dataset.kfold)
# Train them
ensembler.models_tester()


In [4]:
ensembler.metrics

,Test_score,Train_score,Test_score_drop,Accuracy,Precision,Recall,F1_score,Confusion_matrix
"LogisticRegression(max_iter=300, n_jobs=-1, random_state=42)",0.922305,0.915675,0.00724,0.922305,0.617021,0.131818,0.217228,"[[2452, 18], [191, 29]]"
"LogisticRegression(n_jobs=-1, random_state=42)",0.920818,0.916233,0.005004,0.920818,0.574468,0.122727,0.202247,"[[2450, 20], [193, 27]]"


In [5]:
path_to_save = fo.path_to_folder(2, "models") + "first_attempt"
ensembler.models_saver(path_to_save)

'Succesfully saved'

In [6]:
my_model = mo.ml_model(log1)
my_model.load_data(dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test, features_names, dataset.kfold)
my_model.ml_trainer()
my_model.ml_tester()

In [7]:
my_model.model_saver(fo.path_to_folder(2, "models") + "second_attempt")

'Succesfully saved'

In [8]:
os.listdir(fo.path_to_folder(2, "models"))

['model_comparison_noscale_nobalance.csv',
 'first_attempt',
 'second_attempt.pkl']

In [9]:
loaded_model = joblib.load(fo.path_to_folder(2, "models") + "second_attempt.pkl")
loaded_model

LogisticRegression(n_jobs=-1, random_state=42)

# Let's prepare the data and choose a model

# 1) Prepare data for ml models
# For this, we have an useful method in our dataset object. We'll try a basic split and validation, without making any further change to the data (for this attempt)
dataset.model_data(split = .2, cv = 5, epochs = 1)

# 2) Choose model
model = LogisticRegression(n_jobs = -1, max_iter = 300)

# 3) We create a ml_model object and load the data
my_model = md.ml_model(model)

# Load data
my_model.load_data(dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test, features_names, dataset.kfold)

# 4) We train the model
my_model.ml_trainer()

my_model.model.intercept_

path_to_save = fo.path_to_folder(2, "models") + "First_attempt.pkl"
joblib.dump(my_model.model, path_to_save)

loaded_model = joblib.load(path_to_save)
loaded_model

loaded_model.intercept_